In [ ]:
##### Beginning of file

# This file was generated by PredictMD version 0.21.0-DEV
# For help, please visit https://www.predictmd.net

import PredictMD

### Begin project-specific settings

PredictMD.require_julia_version("v0.7.0")

PredictMD.require_predictmd_version("0.21.0-DEV")

# PredictMD.require_predictmd_version("0.21.0-DEV", "0.22.0-")

PROJECT_OUTPUT_DIRECTORY = PredictMD.project_directory(
    homedir(),
    "Desktop",
    "breast_cancer_biopsy_example",
    )

### End project-specific settings

### Begin Knet neural network classifier code

display(knet_mlp_classifier_hist_testing)

PredictMD.singlelabelbinaryclassificationmetrics(
    knet_mlp_classifier,
    smoted_training_features_df,
    smoted_training_labels_df,
    single_label_name,
    positive_class;
    sensitivity = 0.95,
    )

PredictMD.singlelabelbinaryclassificationmetrics(
    knet_mlp_classifier,
    testing_features_df,
    testing_labels_df,
    single_label_name,
    positive_class;
    sensitivity = 0.95,
    )

knet_mlp_classifier_filename = joinpath(
    PROJECT_OUTPUT_DIRECTORY,
    "knet_mlp_classifier.jld2",
    )

PredictMD.save_model(knet_mlp_classifier_filename, knet_mlp_classifier)

### End Knet neural network classifier code

##### End of file

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*